In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM

In [ ]:
logger = logging.getLogger(__name__)
global_config = None

In [ ]:
dataset_name = "lamini_docs.jsonl"
dataset_path = "/content/lamini_docs.jsonl"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "google/gemma-3-270m"
token = "YOUR_API_KEY"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=token)
model = AutoModelForCausalLM.from_pretrained(model_id, use_auth_token=token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

2025-08-24 20:31:37.422457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756067497.764251      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756067497.861444      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("Cynaptics/persona-chat")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
for i in range(5):
    print(ds["train"][i])
    print("-" * 50)

{'conv_id': 'conv60796', 'persona_b': ['I would love to try the local food with my friend.', 'i am quiet but confident.', 'I love to watch movies with my dad on a rainy day.', 'i try to limit how much i eat.', 'I just finished practicing my bass guitar in the lifeguard station.'], 'dialogue': ['Persona A: Hey there!', "Persona B: What's your name?", 'Persona A: My name is William.', 'Persona B: Nice to meet you Gavin. What kind of movies do you like to watch?', 'Persona A: I like to watch movies that make me feel something. I like to be able to relate to the characters and feel their emotions.', 'Persona B: I can relate to that. I like to watch movies that make me think about things in a different way.', "Persona A: That's a good way to put it. I like to be challenged by movies.", 'Persona B: What are some of your favorite movies?', 'Persona A: I have a lot of favorite movies, but some of my favorites include "The Shawshank Redemption," "The Godfather," and "The Lord of the Rings."', '

In [ ]:
def flatten_dataset(ds):
    """Flattens the nested structure of the dataset."""
    flattened_ds = []
    for entry in ds:
        flattened_ds.append({
            "conv_id": entry["conv_id"],
            "persona_b": entry["persona_b"],
            "dialogue": entry["dialogue"],
            "reference": entry["reference"],
            "__index_level_0__": entry["__index_level_0__"]
        })
    return flattened_ds

# Flatten the training split of the dataset
flattened_train_ds = flatten_dataset(ds["train"])

# Convert to a Hugging Face Dataset object
from datasets import Dataset
flattened_train_ds = Dataset.from_list(flattened_train_ds)

print("Flattened dataset example:")
print(flattened_train_ds[0])

Flattened dataset example:
{'conv_id': 'conv60796', 'persona_b': ['I would love to try the local food with my friend.', 'i am quiet but confident.', 'I love to watch movies with my dad on a rainy day.', 'i try to limit how much i eat.', 'I just finished practicing my bass guitar in the lifeguard station.'], 'dialogue': ['Persona A: Hey there!', "Persona B: What's your name?", 'Persona A: My name is William.', 'Persona B: Nice to meet you Gavin. What kind of movies do you like to watch?', 'Persona A: I like to watch movies that make me feel something. I like to be able to relate to the characters and feel their emotions.', 'Persona B: I can relate to that. I like to watch movies that make me think about things in a different way.', "Persona A: That's a good way to put it. I like to be challenged by movies.", 'Persona B: What are some of your favorite movies?', 'Persona A: I have a lot of favorite movies, but some of my favorites include "The Shawshank Redemption," "The Godfather," and "

In [ ]:
def combine_persona_dialogue(entry):
    """Combines the persona and dialogue into a single text field."""
    persona_text = " ".join(entry["persona_b"])
    dialogue_text = " ".join(entry["dialogue"])
    entry["text"] = f"persona: {persona_text} dialogue: {dialogue_text}"
    return entry

# Apply the combination function to the flattened dataset
combined_train_ds = flattened_train_ds.map(combine_persona_dialogue)

print("\nCombined dataset example:")
print(combined_train_ds[0]["text"])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]


Combined dataset example:
persona: I would love to try the local food with my friend. i am quiet but confident. I love to watch movies with my dad on a rainy day. i try to limit how much i eat. I just finished practicing my bass guitar in the lifeguard station. dialogue: Persona A: Hey there! Persona B: What's your name? Persona A: My name is William. Persona B: Nice to meet you Gavin. What kind of movies do you like to watch? Persona A: I like to watch movies that make me feel something. I like to be able to relate to the characters and feel their emotions. Persona B: I can relate to that. I like to watch movies that make me think about things in a different way. Persona A: That's a good way to put it. I like to be challenged by movies. Persona B: What are some of your favorite movies? Persona A: I have a lot of favorite movies, but some of my favorites include "The Shawshank Redemption," "The Godfather," and "The Lord of the Rings." Persona B: Those are all great movies! I love "The

In [ ]:
def tokenize_function(examples):
    """Tokenizes the text data."""
    return tokenizer(examples["text"], truncation=True, max_length=512)

# Apply the tokenization function to the combined dataset
tokenized_train_ds = combined_train_ds.map(tokenize_function, batched=True)

print("\nTokenized dataset example (input_ids):")
print(tokenized_train_ds[0]["input_ids"])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]


Tokenized dataset example (input_ids):
[2, 89334, 236787, 564, 1093, 2765, 531, 2056, 506, 2263, 2780, 607, 1041, 4389, 236761, 858, 1006, 12010, 840, 12081, 236761, 564, 2765, 531, 4526, 13154, 607, 1041, 15030, 580, 496, 44808, 1719, 236761, 858, 2056, 531, 4576, 1217, 1623, 858, 9039, 236761, 564, 1164, 8585, 36419, 1041, 17840, 15583, 528, 506, 225279, 6410, 236761, 22986, 236787, 119997, 562, 236787, 29328, 993, 236888, 119997, 603, 236787, 2900, 236789, 236751, 822, 1463, 236881, 119997, 562, 236787, 3551, 1463, 563, 8966, 236761, 119997, 603, 236787, 32988, 531, 2874, 611, 83654, 236761, 2900, 2712, 529, 13154, 776, 611, 1133, 531, 4526, 236881, 119997, 562, 236787, 564, 1133, 531, 4526, 13154, 600, 1386, 786, 2597, 2613, 236761, 564, 1133, 531, 577, 2735, 531, 23194, 531, 506, 7579, 532, 2597, 910, 19979, 236761, 119997, 603, 236787, 564, 740, 23194, 531, 600, 236761, 564, 1133, 531, 4526, 13154, 600, 1386, 786, 1751, 1003, 2432, 528, 496, 1607, 1595, 236761, 119997, 562, 2367

In [ ]:
# Format the dataset to PyTorch tensors
tokenized_train_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

print("\nFormatted dataset example:")
print(tokenized_train_ds[0])


Formatted dataset example:
{'input_ids': tensor([     2,  89334, 236787,    564,   1093,   2765,    531,   2056,    506,
          2263,   2780,    607,   1041,   4389, 236761,    858,   1006,  12010,
           840,  12081, 236761,    564,   2765,    531,   4526,  13154,    607,
          1041,  15030,    580,    496,  44808,   1719, 236761,    858,   2056,
           531,   4576,   1217,   1623,    858,   9039, 236761,    564,   1164,
          8585,  36419,   1041,  17840,  15583,    528,    506, 225279,   6410,
        236761,  22986, 236787, 119997,    562, 236787,  29328,    993, 236888,
        119997,    603, 236787,   2900, 236789, 236751,    822,   1463, 236881,
        119997,    562, 236787,   3551,   1463,    563,   8966, 236761, 119997,
           603, 236787,  32988,    531,   2874,    611,  83654, 236761,   2900,
          2712,    529,  13154,    776,    611,   1133,    531,   4526, 236881,
        119997,    562, 236787,    564,   1133,    531,   4526,  13154,    600

In [ ]:
training_args = TrainingArguments(
    output_dir=tempfile.mkdtemp(),  # Directory for saving model checkpoints and logs
    num_train_epochs=1,  # Number of training epochs
    per_device_train_batch_size=2,  # Reduced batch size
    gradient_accumulation_steps=2, # Accumulate gradients over 2 steps
    learning_rate=2e-5,  # Learning rate
    logging_dir=tempfile.mkdtemp(),  # Directory for storing logs
    report_to="none" # Disable Weights & Biases logging
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    data_collator=data_collator,
)

print(trainer)

In [ ]:
trainer.train()

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
500,0.868500
1000,0.756200
1500,0.691200
2000,0.649400
2500,0.626300


TrainOutput(global_step=2500, training_loss=0.7183256469726562, metrics={'train_runtime': 2617.1849, 'train_samples_per_second': 7.642, 'train_steps_per_second': 0.955, 'total_flos': 4343127679586304.0, 'train_loss': 0.7183256469726562, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./my_finetuned_model")
tokenizer.save_pretrained("./my_finetuned_model")

('./my_finetuned_model/tokenizer_config.json',
 './my_finetuned_model/special_tokens_map.json',
 './my_finetuned_model/tokenizer.model',
 './my_finetuned_model/added_tokens.json',
 './my_finetuned_model/tokenizer.json')